Modelo 1 de ML:

El objetivo es que al ingresar un indice de una obra del Maestro Fernando Botero, aparezca el cuadro con la mayor similitud usando el modelo de ML del coseno de similitud. Al obtener el cuadro más similar podremos saber un precio aproximado usando los precios de venta en Christie´s.

In [26]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import statsmodels.api as sm

#Traemos el dataset listo para el modelo
df_botero=pd.read_excel(r"C:\Users\param\OneDrive\Escritorio\Programacion\Botero\Datos_modelo_botero.xlsx")
#Eliminamos la variable venta ya que la idea es explicarla. Las variables como Nombre, URL y Area, afectan para el modelo. Por eso se quitan.
df_modelo_coseno=df_botero.drop(columns=['Nombre cuadro', 'Venta','URL',"Área"],axis=1)

In [19]:
#Crear la fecha en dummies
columnas = ['50-59', '60-69', '70-79', '80-89',
            '90-99', '2000-09', '2010-2019']

df_modelo_coseno[columnas] = df_modelo_coseno[columnas].applymap(
    lambda x: 1 if pd.notnull(x) and x != 0 else 0
)

C:\Users\param\AppData\Local\Temp\ipykernel_20604\3687738000.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_modelo_coseno[columnas] = df_modelo_coseno[columnas].applymap(


In [20]:
# Calcular la matriz de similitud
similarity_matrix = cosine_similarity(df_modelo_coseno)

# Para mejor visualización
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=df_botero.index,  # Usar el índice original 
    columns=df_botero.index
)


In [21]:
#Creamos la funcion de recomendación 
def recomendar_obras(indice_obra, similarity_df, df_info, n=5, mostrar_columnas=None):
    """
    Retorna las n obras más similares a una obra dada según la matriz de similitud.

    Parámetros:
    - indice_obra: índice de la obra base.
    - similarity_df: matriz de similitud (DataFrame cuadrado).
    - df_info: DataFrame con información de las obras (ej. df_botero).
    - n: número de recomendaciones.
    - mostrar_columnas: lista de columnas opcional para mostrar (ej. ['Título', 'Autor', 'Venta']).

    Retorna:
    - DataFrame con las obras más similares.
    """
    if indice_obra not in similarity_df.index:
        raise ValueError("El índice dado no está en la matriz de similitud.")

    vector_similitud = similarity_df.loc[indice_obra]
    vector_ordenado = vector_similitud.drop(indice_obra).sort_values(ascending=False)
    indices_similares = vector_ordenado.head(n).index

    if mostrar_columnas:
        return df_info.loc[indices_similares, mostrar_columnas]
    else:
        return df_info.loc[indices_similares]


In [22]:
indice_obra = 1 # Acá se digita la obra que se desea encontrar la de mayor similitud

print("Obra de referencia:", df_botero.loc[indice_obra, ['Nombre cuadro', 'URL']])

recomendar_obras(
    indice_obra=indice_obra,
    similarity_df=similarity_df,
    df_info=df_botero,
    n=5,
    mostrar_columnas=['Nombre cuadro', 'Venta', 'URL']
)


Obra de referencia: Nombre cuadro                                   The Musicians
URL              https://www.christies.com/en/lot/lot-6453127
Name: 1, dtype: object


,Nombre cuadro,Venta,URL
50,Tablao flamenco,2055000,https://www.christies.com/en/lot/lot-6236660
0,The Playroom,3680000,https://www.christies.com/en/lot/lot-6509437
32,The Bather,1830000,https://www.christies.com/en/lot/lot-6317504
17,Man Eating,1071000,https://www.christies.com/en/lot/lot-6444057
3,Shoeshine,1171800,https://www.christies.com/en/lot/lot-6523479


MODELO 2- Observemos los pesos de cada variable con una regresión lineal que explique la variable Precio.

In [23]:
#Eliminamos las variables que no usaremos
df_regresion_lineal=df_botero.drop(columns=['Nombre cuadro','URL',"Área"],axis=1)

In [24]:
# 1. Variable dependiente (precio de venta)
y = df_regresion_lineal['Venta']

# 2. Variables independientes (todas las dummies)
X = df_regresion_lineal.drop(columns=['Venta'])

# 3. Agregar constante (intercepto)
X = sm.add_constant(X)

# 4. Ajustar el modelo
modelo = sm.OLS(y, X).fit()

# 5. Ver resultados
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                  Venta   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     3.042
Date:                Fri, 04 Jul 2025   Prob (F-statistic):            0.00765
Time:                        12:22:49   Log-Likelihood:                -723.26
No. Observations:                  51   AIC:                             1513.
Df Residuals:                      18   BIC:                             1576.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    1

In [27]:
#Eliminemos las variables temáticas a excepción de Grupo de figuras por tener una alta correlación con la variable Precio
df_regresion_lineal_2=df_botero.drop(columns=['1 figura',
       '2 Figuras', 'Bodegon', 'Animales',
       'Escenas urbanas/figuras', 'Escenas rurales/Figuras',
       'Escena hogar/ figuras', 'Historico', 'Politico', 'Religion',
       'Desnudo/ Erotico', 'Paisaje', 'Personal', 'Carbon',
       'Busqueda(sin Boterismo)','Nombre cuadro','URL',"Área"],axis=1)

In [28]:
# 1. Variable dependiente (precio de venta)
y = df_regresion_lineal_2['Venta']

# 2. Variables independientes (todas las dummies)
X = df_regresion_lineal_2.drop(columns=['Venta'])

# 3. Agregar constante (intercepto)
X = sm.add_constant(X)

# 4. Ajustar el modelo
modelo = sm.OLS(y, X).fit()

# 5. Ver resultados
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                  Venta   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.633
Method:                 Least Squares   F-statistic:                     6.084
Date:                Fri, 04 Jul 2025   Prob (F-statistic):           5.04e-06
Time:                        12:24:05   Log-Likelihood:                -734.44
No. Observations:                  51   AIC:                             1505.
Df Residuals:                      33   BIC:                             1540.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -2

MODELO 3- Creamos en streamlit un modelo para que dado cualquier cuadro que se presente, aparezca el de mayor similitud de acuerdo a los atributos que tenga el cuadro.

In [29]:
#Creamos el diccionar de entrada

entrada_usuario = {
    # --- Décadas ---
    '50-59': 0,
    '60-69': 0,
    '70-79': 1,
    '80-89': 0,
    '90-99': 0,
    '2000-09': 0,
    '2010-2019': 0,

    # --- Temática ---
    '1 figura': 0,
    '2 Figuras': 0,
    'Grupo de figuras': 0,
    'Bodegon': 0,
    'Animales': 0,
    'Escenas urbanas/figuras': 0,
    'Escenas rurales/Figuras': 0,
    'Escena hogar/ figuras': 0,
    'Historico': 0,
    'Politico': 0,
    'Religion': 0,
    'Desnudo/ Erotico': 0,
    'Paisaje': 1,
    'Personal': 0,
    'Carbon': 0,
    'Busqueda(sin Boterismo)': 0,

    # --- Tamaño ---
    'tamaño_colección': 0,
    'tamaño_grande': 1,
    'tamaño_mediano-grande': 0,
    'tamaño_mediano-pequeño': 0,
    'tamaño_pequeño': 0,

    # --- Paso por galería NY ---
    'Marlborough NY_paso': 0,

    # --- Referencias literarias ---
    'Mas de 4 literaturas': 1,

    #---Paso por 4 exhibiciones---
    'Mas de 4 exhibiciones':1,

    #---Mas de 4 coleccionistas---
    'Mas de 4 coleccionistas':1

}

In [30]:
# Creamos la función para filtrar 

def filtrar_por_entrada(entrada_usuario, df_botero):
    df_filtrado = df_botero.copy()
    df_coincidencia_exacta = df_filtrado.copy()
    df_mas_cercanos = df_filtrado.copy()

    print(f"Total inicial: {len(df_filtrado)} registros")

    # --- 1. Filtro por TAMAÑO ---
    tamaños = [
        'tamaño_colección', 'tamaño_grande', 'tamaño_mediano-grande',
        'tamaño_mediano-pequeño', 'tamaño_pequeño'
    ]
    for tam in tamaños:
        if entrada_usuario.get(tam, 0) == 1:
            df_coincidencia_exacta = df_filtrado[df_filtrado[tam] == 1]
            if not df_coincidencia_exacta.empty:
                df_mas_cercanos = df_coincidencia_exacta.copy()
            print(f"Filtrado por tamaño: {tam} → {len(df_coincidencia_exacta)} resultados")
            break

    # --- 2. Más de 4 exhibiciones ---
    if entrada_usuario.get('Mas de 4 exhibiciones', 0) == 1:
        df_coincidencia_exacta = df_coincidencia_exacta[df_coincidencia_exacta['Mas de 4 exhibiciones'] == 1]
        if not df_coincidencia_exacta.empty:
            df_mas_cercanos = df_coincidencia_exacta.copy()
        print(f"Filtrado por más de 4 exhibiciones → {len(df_coincidencia_exacta)} resultados")

    # --- 3. Más de 4 literaturas ---
    if entrada_usuario.get('Mas de 4 literaturas', 0) == 1:
        df_coincidencia_exacta = df_coincidencia_exacta[df_coincidencia_exacta['Mas de 4 literaturas'] == 1]
        if not df_coincidencia_exacta.empty:
            df_mas_cercanos = df_coincidencia_exacta.copy()
        print(f"Filtrado por más de 4 literaturas → {len(df_coincidencia_exacta)} resultados")

    # --- 4. Marlborough NY ---
    if entrada_usuario.get('Marlborough NY_paso', 0) == 1:
        df_coincidencia_exacta = df_coincidencia_exacta[df_coincidencia_exacta['Marlborough NY_paso'] == 1]
        if not df_coincidencia_exacta.empty:
            df_mas_cercanos = df_coincidencia_exacta.copy()
        print(f"Filtrado por paso por Marlborough NY → {len(df_coincidencia_exacta)} resultados")

    # --- 5. Década ---
    decadas = ['50-59', '60-69', '70-79', '80-89', '90-99', '2000-09', '2010-2019']
    for dec in decadas:
        if entrada_usuario.get(dec, 0) == 1:
            df_coincidencia_exacta = df_coincidencia_exacta[df_coincidencia_exacta[dec] == 1]
            if not df_coincidencia_exacta.empty:
                df_mas_cercanos = df_coincidencia_exacta.copy()
            print(f"Filtrado por década: {dec} → {len(df_coincidencia_exacta)} resultados")
            break

    # --- 6. Temática ---
    tematicas = [
        '1 figura', '2 Figuras', 'Grupo de figuras', 'Bodegon', 'Animales',
        'Escenas urbanas/figuras', 'Escenas rurales/Figuras',
        'Escena hogar/ figuras', 'Historico', 'Politico', 'Religion',
        'Desnudo/ Erotico', 'Paisaje', 'Personal', 'Carbon', 'Busqueda(sin Boterismo)'
    ]
    for tema in tematicas:
        if entrada_usuario.get(tema, 0) == 1:
            df_coincidencia_exacta = df_coincidencia_exacta[df_coincidencia_exacta[tema] == 1]
            if not df_coincidencia_exacta.empty:
                df_mas_cercanos = df_coincidencia_exacta.copy()
            print(f"Filtrado por temática: {tema} → {len(df_coincidencia_exacta)} resultados")
            break

    columnas_a_mostrar = ['Nombre cuadro', 'URL', 'Venta']

    # --- Si hay coincidencias exactas ---
    if not df_coincidencia_exacta.empty:
        print(f"\n✅ Estas obras tienen las mismas condiciones exactas. Estos son sus índices:")
        df_coincidencia_exacta = df_coincidencia_exacta[columnas_a_mostrar].copy()
        df_coincidencia_exacta['Índice'] = df_coincidencia_exacta.index
        print(df_coincidencia_exacta[['Índice'] + columnas_a_mostrar])
        return df_coincidencia_exacta['Índice'].tolist()

    # --- Si no hay coincidencias exactas, mostrar las más similares ---
    print(f"\n⚠️ No hay un cuadro con todas las condiciones, pero estos son los más parecidos:\n")
    df_mas_cercanos = df_mas_cercanos[columnas_a_mostrar].copy()
    df_mas_cercanos['Índice'] = df_mas_cercanos.index

    for _, fila in df_mas_cercanos.iterrows():
        print(f"Índice: {fila['Índice']}")
        print(f"  🎨 Nombre: {fila['Nombre cuadro']}")
        print(f"  🔗 URL: {fila['URL']}")
        print(f"  💰 Venta: {fila['Venta']}\n")

    return df_mas_cercanos['Índice'].tolist()


In [31]:
resultados = filtrar_por_entrada(entrada_usuario, df_botero)
print("Índices recomendados:", resultados)


Total inicial: 51 registros
Filtrado por tamaño: tamaño_grande → 11 resultados
Filtrado por más de 4 exhibiciones → 2 resultados
Filtrado por más de 4 literaturas → 2 resultados
Filtrado por década: 70-79 → 0 resultados
Filtrado por temática: Paisaje → 0 resultados

⚠️ No hay un cuadro con todas las condiciones, pero estos son los más parecidos:

Índice: 1
  🎨 Nombre: The Musicians
  🔗 URL: https://www.christies.com/en/lot/lot-6453127
  💰 Venta: 5132000

Índice: 50
  🎨 Nombre: Tablao flamenco
  🔗 URL: https://www.christies.com/en/lot/lot-6236660
  💰 Venta: 2055000

Índices recomendados: [1, 50]
